# Начальная настройка

In [6]:
# Ячейка 1: Импорты (новый синтаксис)
import pandas as pd
import xgboost as xgb
from langchain_openai import ChatOpenAI
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.embeddings import HuggingFaceEmbeddings
import sys
import os
import json
from qgis.core import QgsApplication, QgsVectorLayer, QgsFeature, QgsProject
from qgis.PyQt.QtCore import QVariant 

print("Библиотеки импортированы!")

Библиотеки импортированы!


In [10]:
# Ячейка 2: Настройка окружения (безопасная инициализация)
# Эта ячейка проверяет состояние окружения и инициализирует компоненты только один раз

# Глобальная переменная для отслеживания инициализации
if 'qgs' not in globals() or qgs is None:
    print("Инициализация QGIS...")
    QgsApplication.setPrefixPath("/opt/anaconda3/envs/pyqgis_env/bin", True)
    qgs = QgsApplication([], False)
    qgs.initQgis()
    print("QGIS инициализирован")
else:
    print("QGIS уже инициализирован")

# Проверка других компонентов


print("\n✅ Окружение готово к работе!")


QGIS уже инициализирован

✅ Окружение готово к работе!


In [ ]:
# Ячейка 3: Загрузка данных (с проверкой на повторную загрузку)
if 'data' not in globals() or data is None:
    print("Загрузка данных...")
    data = pd.read_csv(
        "/Users/rodionduktanov/anaconda_projects/RAG_Caspian_Analysis/ЦК(25.06.25)/pars_test.csv",
        low_memory=False  # Убираем предупреждение о mixed types
    )
    print(f"Данные загружены: {len(data)} строк, {len(data.columns)} колонок")
else:
    print(f"Данные уже загружены: {len(data)} строк, {len(data.columns)} колонок")

data

Данные уже загружены: 18103 строк, 226 колонок


,layer,id,lon,lat,Cорг_Сорг,Пункты замеров _Наиме,Пункты замеров _HF,Пункты замеров _Глуби,Пункты замеров _Источ,Пункты замеров _Год з�,...,очаг генерации_Зр-ть,Загрязнители_id,Загрязнители_Загря,Очаг генерац_R0,Очаг генерац_Зр-ть,Профиль ГАУС_Image,Профиль ГАУС_Профиль,Балаханское время_PWD,Сабунчиское время_ PWD,Время перерыва_PWD
0,Cорг,0,9.454725e+06,4.379692e+06,1.25,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0
1,Cорг,1,9.454725e+06,4.379692e+06,0.75,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0
2,Cорг,2,9.616109e+06,4.477144e+06,0.75,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0
3,Cорг,3,9.341950e+06,4.449786e+06,1.75,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0
4,Cорг,4,9.765660e+06,4.355448e+06,1.25,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18098,Время перерыва,51,9.680231e+06,4.134610e+06,1.75,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,50.0
18099,Время перерыва,52,9.398585e+06,4.465175e+06,1.75,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0
18100,Время перерыва,53,9.319804e+06,4.289158e+06,1.75,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0
18101,Время перерыва,54,9.520523e+06,4.226424e+06,1.75,NaN,45.0,NaN,NaN,1988.0,...,NaN,12.5,NaN,NaN,NaN,NaN,NaN,125.0,50.0,200.0


: 

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18103 entries, 0 to 18102
Columns: 226 entries, layer to Время перерыва_PWD
dtypes: float64(98), int64(1), object(127)
memory usage: 31.2+ MB


: 

: 

In [ ]:
data.describe()

,id,lon,lat,Cорг_Сорг,Пункты замеров _HF,Пункты замеров _Глуби,Пункты замеров _Год з�,Изолинии_Толщи,HI_HI,PI_PI,...,Кровля надкирмакинской песчаной свиты (N22nas top)_Глуб.,Кровля надкирмакинской глинистой свиты (N22nac top)_Глуб.,Путь транспортировки нефти_id,Аварийные разливы нефти_id,Пленки нефтепродуктов_id,Взры трубопровода_id,Загрязнители_id,Балаханское время_PWD,Сабунчиское время_ PWD,Время перерыва_PWD
count,18103.000000,1.810300e+04,1.810300e+04,18103.000000,18103.000000,1262.000000,18103.000000,18103.000000,18103.000000,18103.000000,...,18103.000000,18103.000000,18103.000000,18103.000000,18103.000000,18103.0,18103.000000,18103.000000,18103.000000,18103.000000
mean,208.937137,9.481205e+06,4.377235e+06,1.749365,45.393636,-575.471474,1988.034746,200.071811,300.011048,0.425019,...,-5492.349334,-4997.099928,9.999392,3.500000,19.500000,1.0,12.500000,125.060763,50.096669,200.226482
std,209.621268,1.156155e+05,9.344143e+04,0.023199,19.512800,1088.670251,2.347117,11.538167,5.486989,0.003466,...,349.027595,318.785203,0.141607,0.031093,0.502425,0.0,0.252049,5.939904,6.382163,11.843339
min,0.000000,9.113194e+06,4.052409e+06,0.250000,0.000000,-2414.000000,1968.000000,100.000000,50.000000,0.325000,...,-10000.000000,-9500.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000,0.000000,0.000000
25%,59.000000,9.397811e+06,4.318930e+06,1.750000,45.000000,-1716.750000,1988.000000,200.000000,300.000000,0.425000,...,-5500.000000,-5000.000000,10.000000,3.500000,19.500000,1.0,12.500000,125.000000,50.000000,200.000000
50%,153.000000,9.454725e+06,4.379692e+06,1.750000,45.000000,0.000000,1988.000000,200.000000,300.000000,0.425000,...,-5500.000000,-5000.000000,10.000000,3.500000,19.500000,1.0,12.500000,125.000000,50.000000,200.000000
75%,289.000000,9.538899e+06,4.443054e+06,1.750000,45.000000,0.000000,1988.000000,200.000000,300.000000,0.425000,...,-5500.000000,-5000.000000,10.000000,3.500000,19.500000,1.0,12.500000,125.000000,50.000000,200.000000
max,1261.000000,9.905585e+06,5.370233e+06,1.750000,2500.000000,2204.000000,2008.000000,700.000000,550.000000,0.625000,...,1500.000000,1500.000000,16.000000,6.000000,38.000000,1.0,24.000000,500.000000,500.000000,500.000000


: 

: 

In [ ]:

for x in data.columns: 
    print(x)

layer
id
lon
lat
Cорг_Сорг
Пункты замеров _Наиме
Пункты замеров _HF
Пункты замеров _Глуби
Пункты замеров _Источ
Пункты замеров _Год з�
Изолинии_Толщи
HI_HI
PI_PI
R0,%_R0
R0,%_Зр-ть
R0, %_R0
R0, %_Зр-ть
Rо,%_R0
Rо,%_Зр-ть
TR,%_TR
TR, %_TR
Tmax_Tmax
ТR, %_TR
Лист1_Field1
Лист1_Field2
Лист1_Field3
Лист1_Field4
Лист1_Field5
Лист1_Field6
Лист1_Field7
Лист1_Field8
Лист1_Field9
Лист1_Field10
Лист1_Field11
Лист1_Field12
Лист1_Field13
Лист1_Field14
Лист1_Field15
Лист1_Field16
Сорг_Сорг
Области распространения резервуаров_Литол
Эоцен (Pg2)_Толщ.
Голоцен (H)_Толщ.
Литология_PWD
Литология_Фация
Литология_Литол
Литология_Обста
АВПД, МПа_АВПД
Палеоцен (Pg1)_Толщ.
Палеорусла_Палео
Палеорусла_Наиме
Палеорусла_Источ
Батиметрия (Q top)_Глуб.
Профиль ГАУС 1_Профи
Профиль ГАУС 1_Image
Граница ГАУС_ГАУС
Профиль ГАУС 2_Профи
Профиль ГАУС 2_Image
Профиль ГАУС 3_Профи
Профиль ГАУС 3_Image
Кирмакинское время_PWD
Профиль ГАУС 4_Профи
Профиль ГАУС 4_Image
взрыв танкера_id
взрыв танкера_Танке
взрыв танкера_Год
По

: 

: 

In [ ]:
print(data['Взры трубопровода_Взрыв'])

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
18098    NaN
18099    NaN
18100    NaN
18101    NaN
18102    NaN
Name: Взры трубопровода_Взрыв, Length: 18103, dtype: object


: 

: 

In [ ]:
print(data['Взры трубопровода_Взрыв'].value_counts(dropna=False))

NaN                   18102
Взрыв трубопровода        1
Name: Взры трубопровода_Взрыв, dtype: int64


: 

: 